In [1]:
import pandas as pd
import utils
import numpy as np
import warnings
import random
from tqdm import tqdm
import random
from copairs.map import average_precision, mean_average_precision
import logging

logging.basicConfig(format="%(levelname)s:%(asctime)s:%(name)s:%(message)s")
logging.getLogger("copairs").setLevel(logging.INFO)

warnings.simplefilter(action="ignore", category=FutureWarning)

In [2]:
operations = "wellpos_var_mad_int_featselect_harmony_PCA"
batch_size = 20000
null_size = 20000
fdr = 0.1

### Prepare the data

#### Read the ORF parquet file

In [3]:
orf_df = pd.read_parquet(
    f"../profiles/profiles_{operations}.parquet"
)

#### Remove empty wells

In [4]:
orf_df = utils.remove_empty_wells(orf_df)
orf_df.shape

(81660, 385)

#### Remove `poscon` wells.

In [5]:
orf_df = orf_df.query('Metadata_pert_type!="poscon"').reset_index(drop=True)
orf_df.shape

(79730, 385)

#### Remove `BAD CONSTRUCT` wells.

In [6]:
orf_df = orf_df.query('Metadata_broad_sample!="BAD CONSTRUCT"').reset_index(drop=True)
orf_df.shape

(79730, 385)

#### Remove featues with `nan` values.
These need to be removed as the `nan` values will cause the mean average precision calculation to fail.

In [7]:
orf_df = utils.remove_nan_features(orf_df)

Removed nan features: []


### Calculate mAP for each ORF perturbation

In [8]:
# Adding a new column for negative control
orf_df["Metadata_negcon"] = np.where(orf_df["Metadata_pert_type"] == "negcon", 1, 0)

In [9]:
pos_sameby = ["Metadata_JCP2022"]
pos_diffby = []
neg_sameby = ["Metadata_Plate"]
neg_diffby = ["Metadata_negcon"]

In [10]:
metadata_df = utils.get_metadata(orf_df)
feature_df = utils.get_featuredata(orf_df)
feature_values = feature_df.values

In [11]:
result = average_precision(
    metadata_df, feature_values, pos_sameby, pos_diffby, neg_sameby, neg_diffby, batch_size=batch_size
)

INFO:2024-04-04 15:42:19,776:copairs:Indexing metadata...
INFO:2024-04-04 15:42:19,888:copairs:Finding positive pairs...
INFO:2024-04-04 15:42:21,202:copairs:Finding negative pairs...
INFO:2024-04-04 15:42:22,906:copairs:Computing positive similarities...


  0%|          | 0/93 [00:00<?, ?it/s]

INFO:2024-04-04 15:42:28,344:copairs:Computing negative similarities...


  0%|          | 0/63 [00:00<?, ?it/s]

INFO:2024-04-04 15:42:30,667:copairs:Building rank lists...
INFO:2024-04-04 15:42:34,032:copairs:Computing average precision...
INFO:2024-04-04 15:42:34,243:copairs:Creating result DataFrame...
INFO:2024-04-04 15:42:34,256:copairs:Finished.


In [12]:
agg_result = (
    mean_average_precision(result, pos_sameby, null_size=null_size, threshold=fdr, seed=12527)
    .rename(columns={'average_precision': 'mean_average_precision'})
)

INFO:2024-04-04 15:42:34,360:copairs:Computing null_dist...


  0%|          | 0/22 [00:00<?, ?it/s]

INFO:2024-04-04 15:42:34,487:copairs:Computing p-values...


  0%|          | 0/15130 [00:00<?, ?it/s]

In [13]:
agg_result.to_csv(f"output/phenotypic-activity-{operations}.csv.gz", index=False)